In [ ]:
pip install TTS

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 427.7 kB/s  0:02:050:00:0100:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 4.1 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━

In [3]:
pip install torch

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 2.4/241.3 MB 12.2 MB/s eta 0:00:20
    --------------------------------------- 4.7/241.3 MB 12.4 MB/s eta 0:00:20
   - -------------------------------------- 7.3/241.3 MB 11.9 MB/s eta 0:00:20
   - -------------------------------------- 9.7/241.3 MB 12.1 MB/s eta 0:00:20
   -- ------------------------------------- 12.3/241.3 MB 11.9 MB/s eta 0:00:20
   -- ------------------------------------- 14.9/241.3 MB 11.9 MB/s eta 0:00:20
   -- ------------------------------------- 17.3/241.3 MB 11.8 MB/s eta 0:00:19
   --- ------------------------------------ 19.9/241.3 MB 11.9 MB/s eta 0:00:19
   --- ------------------------------------ 22.5/241.3 MB 11.9 MB/s eta 0:00:19
   ---- ----------------------------------- 24.9/241.3 MB 11.9 MB/s eta 0:00:19
   ---- ----------------------------------- 27.3/241.3 MB 11.9 MB/s et

In [4]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
wav = tts.tts(text="Hello world!", speaker_wav="my/cloning/audio.wav", language="en")

ModuleNotFoundError: No module named 'TTS'